In [3]:
%load_ext autoreload
%autoreload 2

# Insert root directory to path
import os
import sys
from pathlib import Path
root_dir = str(Path(os.getcwd()).parent)
if str(root_dir) not in sys.path:
    sys.path.insert(0,str(root_dir))
else:
    print('PATH O.K.')

import numpy as np
import ipyvolume as ipv
import cv2
import matplotlib.pyplot as plt
import kornia

from slib.lie_algebra import random_rot3, random_trans3
from slib.plane3d import Plane3d
from slib.plot_3d import plot_planar_rect, init_3d_plot
from slib.lie_algebra import SE3, SO3
from slib.camera import CameraIntrinsicts, Camera, Cameras


# Draw a single plane
Parts taken from [Multi view notebooks](https://github.com/maxcrous/multiview_notebooks)

In [28]:
!ls -lh

total 432
-rw-r--r--@ 1 shmuelrippa  staff   8.1K Jan 31 13:10 colab_play.ipynb
-rw-r--r--  1 shmuelrippa  staff    83K Jun 30 14:24 covid19-faq.txt
-rw-r--r--  1 shmuelrippa  staff    13K Jul 12 09:04 dl_course_1.ipynb
-rw-r--r--  1 shmuelrippa  staff   8.8K Jul  4 13:05 loss1.pdf
-rw-r--r--  1 shmuelrippa  staff    15K Jul 13 10:22 multiview_geometry.ipynb
-rw-r--r--  1 shmuelrippa  staff    15K Jul  4 13:08 rnn_lstm.ipynb
-rw-r--r--@ 1 shmuelrippa  staff    19K Jun 23 12:54 simple_mlp.ipynb
-rw-r--r--  1 shmuelrippa  staff    21K Jul 11 12:08 transformer.ipynb
-rw-r--r--@ 1 shmuelrippa  staff    13K Jun  2 18:20 useful_libs.ipynb


In [30]:
# Read images
for i in range(11): 
    img = cv2.imread(f'../images/calibration/{i}.jpg')
    img = cv2.resize(img, None, fx=0.25, fy=0.25)
    cv2.imwrite(f'../images/calibration/calib_{i}.jpg', img)

    print(img.shape)

(1000, 1500, 3)
(1000, 1500, 3)
(1000, 1500, 3)
(1000, 1500, 3)
(1000, 1500, 3)
(1000, 1500, 3)
(1000, 1500, 3)
(1000, 1500, 3)
(1000, 1500, 3)
(1000, 1500, 3)
(1000, 1500, 3)


In [4]:

img_dir = Path('/Users/shmuelrippa/dvlp/dvlp/images/calibration')
img_file = img_dir / 'calib_4.jpg'
img = cv2.imread(str(img_file))
# img = cv2.resize(img, None, fx=0.25, fy=0.25)
print(img.shape)

fx = 1452.26601 
fy = 1449.93355
cx = 759.064530
cy = 457.146908
width = 1500
height = 1000

distortions = [-1.51960304e-01,  5.60700273e-01, -1.28234990e-02,  1.41775450e-03, 5.23404322e+00]
ci = CameraIntrinsicts.from_opencv_model(fx,fy,cx,cy,distortions,width, height)


rot_v_00 = np.array([[-0.20247377], [1.40040087], [2.40735099]])
trans_v_00 = np.array([[3.84588435], [3.33279547], [26.53234229]])

rot_v_0 = np.array([-0.20247377, 1.40040087, 2.40735099])
rot_v_1 = np.array([[-0.44448518], [-1.37454204], [-2.33434062]])
rot_v_2 = np.array([[-0.7470287 ], [ 0.8806656 ], [ 1.49779009]])
rot_v_3 = np.array([[-0.03328229], [ 1.26938756], [ 2.70563679]])
rot_v_4 = np.array([[-0.83852022], [-0.84686949], [-1.46605745]])
rot_v_5 = np.array([[-1.01555167], [-0.74624915], [-1.14510058]])
rot_v_6 = np.array([[-0.47130223], [-0.89999994], [-0.56372182]])
rot_v_7 = np.array([[-0.7208132 ], [ 0.53782897], [ 1.16244219]])
rot_v_8 = np.array([[0.07541929],  [1.90962255],  [2.46269658]])
rot_v_9 = np.array([[-0.73442614], [-0.58029994], [-1.21790521]])
rots = [rot_v_00, rot_v_1, rot_v_2, rot_v_3, rot_v_4, rot_v_5, rot_v_6, rot_v_7, rot_v_8, rot_v_9]

trans_v_0 = np.array([3.84588435, 3.33279547, 26.53234229])
trans_v_1 = np.array([[-1.11390426], [ 2.81845125], [28.68867561]])
trans_v_2 = np.array([[ 2.48205121], [-0.50069956], [33.90249133]])
trans_v_3 = np.array([[ 4.16217292], [ 5.91685505], [29.21393509]])
trans_v_4 = np.array([[-3.25634978], [ 4.47575481], [36.116295  ]])
trans_v_5 = np.array([[-6.12994266], [ 3.46228711], [39.69617574]])
trans_v_6 = np.array([[-5.68526331], [-1.18649701], [41.05701536]])
trans_v_7 = np.array([[ 4.03303854], [ 0.32427912], [38.35574254]])
trans_v_8 = np.array([[ 1.94444063], [ 0.78947952], [17.67448071]])
trans_v_9 = np.array([[-0.99470531], [ 6.39366495], [36.54023511]])
ts = [trans_v_00, trans_v_1, trans_v_2, trans_v_3, trans_v_4, trans_v_5, trans_v_6, trans_v_7, trans_v_8, trans_v_9]


# cam_00 = Camera.from_rotvec_trans(ci, rot_v_0, trans_v_0)

ii = 0
camera_list = []
for rot,trans in zip(rots,ts):
    img_file = img_dir / f'calib_{ii}.jpg'
    img = cv2.imread(str(img_file))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 

    rotr = cv2.Rodrigues(rot)[0]
    extrinsic = np.concatenate([rotr, trans], axis=1)
    extrinsic = np.vstack([extrinsic, [[0,0,0,1]]]) 

    center = -rotr.T @ trans
    # center = np.append(center, 1)

    cam_00 = Camera.from_rotvec_trans(ci, rot, trans)
    cam_00.append_image(img)
    camera_list.append(cam_00)

    # print(center.squeeze())
    # print(cam_00.center)

    # print('EX ',extrinsic)
    print(center)
    print(cam_00.center)
    print('err ', np.linalg.norm(cam_00.extrinsics-extrinsic), np.linalg.norm(center.squeeze()-cam_00.center))
    ii += 1
    break
cameras = Cameras(camera_list)


(1000, 1500, 3)
5  9
[[ 10.59621268]
 [-18.68066314]
 [-16.3895284 ]]
[ 10.59621268 -18.68066314 -16.3895284 ]
err  2.1677797545656835e-16 3.972054645195637e-15


In [66]:
pw = np.array([0.5, 0.5, 0.5, 1.0])
# p2d, disparity = cam_00.project_3d(pw)
# print(p2d, disparity)

pix, disparity = cam_00.project_point(pw)
print(pix,disparity)

p3d = cam_00.reproject_point(pix, disparity)
print(p3d)

print(np.linalg.norm(pw-p3d))

pc un-distorted  [0.11919842 0.13499647]
pc distorted  [0.11837612 0.13359721]
[930.97814338 650.85398399] 0.03697704041775363
pc distorted  [0.11837612 0.13359721]
xxxxxxx  [0.11837612 0.13359721] [0.11919495 0.13499051] [0.11919495 0.13499051]
pc un-distorted  [0.11919495 0.13499051]
[0.50005117 0.50010707 0.49985626 1.        ]
0.0001864006583065566


In [14]:
p3d = np.array([pw[:3].tolist()])
print(cam_00.K.shape)
print(p3d.shape)
pp, _ =  cv2.projectPoints(pw[:3],
                            rot,
                            trans,
                            cam_00.K[:3,:3],
                            np.array(distortions))
print(pp)

(4, 4)
(1, 3)
[[[930.97814338 650.85398399]]]


In [139]:
print(cam_00.extrinsics_matrix())

[[-0.92944893 -0.36550053  0.05033934  3.84588435]
 [ 0.22443503 -0.45181013  0.86342141  3.33279547]
 [-0.29283716  0.81380401  0.50196556 26.53234229]
 [ 0.          0.          0.          1.        ]]


In [133]:
cam_00 = camera_list[1]

print(cam_00.center)

xmin = cam_00.center[0] - 2.0
xmax = cam_00.center[0] + 2.0
ymin = cam_00.center[1] - 2.0
ymax = cam_00.center[1] + 2.0
zmin = cam_00.center[2] - 2.0
zmax = cam_00.center[2] + 2.0
fig = init_3d_plot(xmin, ymin, xmax, ymax, zmin, zmax)
ipv =cam_00.plot(1.0, show_image=True)
ipv.show()

[-13.62505096 -19.90820892 -15.81915808]


In [134]:
ipv = cameras.plot(1.0)
ipv.show()

In [ ]:
def extrinsics_from_calibration(rotation_vectors, translation_vectors):
    """ Calculates extrinsic matrices from calibration output. 
        
    Args: 
        rotation_vectors (list[np.ndarray]): Rodrigues rotation vectors.
        translation_vectors (list[np.ndarray]): Translation vectors.
    Returns: 
        extrinsics (list[np.ndarray]): A list of camera extrinsic matrices.
            These matrices are 4x4 full-rank.
    """
    
    rotation_matrices = list() 
    for rot in rotation_vectors:
        rotation_matrices.append(cv2.Rodrigues(rot)[0]) 

    extrinsics = list()
    for rot, trans in zip(rotation_matrices, translation_vectors): 
        extrinsic = np.concatenate([rot, trans], axis=1)
        extrinsic = np.vstack([extrinsic, [[0,0,0,1]]]) 
        extrinsics.append(extrinsic)
    
    return extrinsics

def camera_centers_from_extrinsics(extrinsics):
    """ Calculates camera centers from extrinsic matrices. 
    
    Args: 
        extrinsics (list[np.ndarray]):  A list of camera extrinsic matrices.
    Returns: 
        camera_centers (list[np.ndarray]): Homogenous coordinates of camera centers in 
            3D world coordinate frame.
    """
    camera_centers = list() 

    for extrinsic in extrinsics:
        rot = extrinsic[:3, :3]
        trans = extrinsic[:3, 3]
        center = -rot.T @ trans
        center = np.append(center, 1)
        camera_centers.append(center)
    
    return camera_centers

In [63]:
from easydict import EasyDict as edict
import json

def read(path, read_mode):
    with Path(path).open(read_mode) as fin:
        buffer = fin.read()
    return buffer

def read_json_file(path):
    json_str = read(str(path), 'r')
    json_dict = json.loads(json_str)
    return json_dict


point_cloud_root_path = Path('/opt/datasets/DMP_point_cloud')
metadata_file = point_cloud_root_path / 'Metadata.json'
medatata = edict(read_json_file(metadata_file))

ero_cameras =  medatata['cameradata']
camera_names = [c['name'] for c in ero_cameras]

c1 = edict(ero_cameras[0])
print(c1)
K1 = [
    [c1.focalpoint, 0.0,           c1.cx,  0.0],
    [0.0,           c1.focalpoint, c1.cy,  0.0],
    [0.0,           0.0,           0.0, 1.0]

]
print(ero_cameras[0])

{'name': '01', 'width': '2400', 'height': '2000', 'focalpoint': '1779.770796', 'cx': '1201.783555', 'cy': '968.960122'}
{'name': '01', 'width': '2400', 'height': '2000', 'focalpoint': '1779.770796', 'cx': '1201.783555', 'cy': '968.960122'}


In [2]:
p0 = np.array([0.0,0.0,0.0])
p1 = np.array([0.0,1.0,0.0])
p2 = np.array([0.0,0.5,0.5])
plane_3d = Plane3d.from_3_points(p0,p1,p2)


In [40]:
ipv = plot_planar_rect(plane_3d)
ipv.show()

[0.5 0.  0. ]


In [2]:
nx, ny = (2,2)
x = np.linspace(0, 1, nx)
y = np.linspace(0, 1, ny)
X, Y = np.meshgrid(x, y)
# for i in range(nx):
#     for j in range(ny):
#         print(X[j,i], Y[j,i])

Z = 0.5*X + 0.5
ipv.figure()
ipv.plot_surface(X, Z, Y, color="orange")

# fig = ipv.pylab.figure(figsize=(15, 15), width=800)
ipv.xlim(-2.0, 2.0)
ipv.ylim(-2.0, 2.0)
ipv.zlim(-2.0, 2.0)
ipv.pylab.view(azimuth=40, elevation=-150)

ipv.show()

In [3]:
ipv.show()

In [47]:
fx = 600
fy = 600
cx = 1280 /2
cy = 720 /2
K = np.array(
    [
        [fx, 0.0, cx],
        [0.0, fy, cy],
        [0.0,0.0,1.0]
    ]
)
print(K)
R = random_rot3()
t = random_trans3()
print(t.shape)
extrinsic = np.vstack((R, t.T))
print(extrinsic.shape)
n = np.array([[0,0,1]])
print(n.shape)
extrinsic = np.c_[ extrinsic, np.eye(3) ] 
print('ee ',extrinsic.shape)

e = np.concatenate([R, t], axis=1)
e = np.vstack([e, [[0,0,0,1]]]) 
print(e.shape)
print(np.linalg.norm(e-extrinsic))

camera_center = -R.T @ t

print(R)
print(t)
print(camera_center)

[[600.   0. 640.]
 [  0. 600. 360.]
 [  0.   0.   1.]]
(3, 1)
(4, 3)
(1, 3)


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 4 and the array at index 1 has size 3

In [ ]:
def init_3d_plot(box_3d, plane_3d):
    """ Initializes a ipyvolume 3d plot and centers the 
        world view around the center of the plane.
        
    Returns: 
        fig (ipyvolume.pylab.figure): A 3D plot. 
    """
    plane_3d_x_center = plane_3d.x
    plane_3d_y_center = plane_3d.y

    fig = ipv.pylab.figure(figsize=(15, 15), width=800)
    ipv.xlim(box_3d.xmin, bx_3d.xmax)
    ipv.ylim(box_3d.ymin, box3d.ymax)
    ipv.zlim(box_3d.zmin, box3d.zmax)
    ipv.pylab.view(azimuth=40, elevation=-150)
    return fig